In [1]:
import re
import pandas as pd
import numpy as np

from GeralModelY_ import *
from pathlib import Path
from gurobipy import read

pd.set_option('display.max_rows', None)

In [2]:
wilmer = GeralModelY(
    path_dem='/home/wilmer/Documentos/Codes/tesis/Instancias/reales/instancia1/demanda.csv',
    path_preco='/home/wilmer/Documentos/Codes/tesis/Instancias/reales/instancia1/preco.csv',
    path_rota1='/home/wilmer/Documentos/Codes/tesis/Instancias/reales/instancia1/rota.csv',
    Q={'P':561, 'Z':561},
    perio=0,
    nome='instancia1',
    abordagem='T1',
    g_param={
        't': 0, 
        'y': 5,
        'x': 11,
        'ng': 4,
        'attrs': ['Authorizations[Y]','Assignments[X]','Demanda','Preco'],
        }
)

model = wilmer.behavioral_complete_model()

Set parameter Username
Academic license - for non-commercial use only - expires 2025-06-25
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (linux64 - "Ubuntu 24.04.2 LTS")

CPU model: AMD Ryzen 5 2500U with Radeon Vega Mobile Gfx, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Presolve removed 62254 rows and 58240 columns
Presolve time: 0.05s
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (linux64 - "Ubuntu 24.04.2 LTS")

CPU model: AMD Ryzen 5 2500U with Radeon Vega Mobile Gfx, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 63340 rows, 59127 columns and 176986 nonzeros
Model fingerprint: 0x4fd22ac4
Variable types: 0 continuous, 59127 integer (3318 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+05]
  Objective range  [6e+00, 3e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 2e+05]
Presolve removed 62254 row

# pruebas manuales

In [ ]:
# Leer el modelo LP
modelop = read("/home/wilmer/Documentos/Codes/tesis/Instancias/test/test1/test1.lp")  # Asegúrate de que esté en el mismo directorio

# Asegurar que el modelop está actualizado
modelop.update()

# Extraer matriz de restricciones
A = modelop.getA()

# Extraer nombres de variables (columnas de A)
variables = [v.VarName for v in modelop.getVars()]

# Extraer nombres de restricciones (filas de A)
restricciones = [c.ConstrName for c in modelop.getConstrs()]

# Extraer lado derecho (b)
b = [c.RHS for c in modelop.getConstrs()]

# Convertir la matriz dispersa a DataFrame
A_df = pd.DataFrame.sparse.from_spmatrix(A, index=restricciones, columns=variables)

# Convertir b a Series
b_series = pd.Series(b, index=restricciones, name='b')

# Mostrar resultado
A_df

# print("\nVector b (lado derecho):")
# print(b_series)

# print("\nVector x (variables):")
# print(pd.Series(variables, name='x'))


In [ ]:
# True si el valor ∈ {0, -1, 1}
en_conjunto = A_df.isin([0, -1, 1])

# True si TODA la fila está en {0,-1, 1}
fila_toda_en_conjunto = en_conjunto.all(axis=1)

# Filas que tienen al menos un valor FUERA del conjunto
filas_con_valor_distinto = ~fila_toda_en_conjunto

# Índices que cumplen la condición
indicesw = A_df.index[filas_con_valor_distinto]

indicesw, len(indicesw)

In [ ]:
def parse_constraints(lp_path):
    """
    Lee un archivo .lp y devuelve un conjunto con los nombres de las restricciones
    que aparecen en la sección Subject To (o s.t.).
    """
    constraints = set()
    in_subject_to = False

    # Expresiones regulares para detectar el inicio y final de la sección
    start_pattern = re.compile(r'^(Subject To|s\.t\.|st)\b', re.IGNORECASE)
    end_pattern   = re.compile(r'^(Bounds|Binaries|Generals|End)\b', re.IGNORECASE)

    # Patrón para capturar el nombre de la restricción: texto antes de los dos puntos
    constr_pattern = re.compile(r'^(\S+)\s*:\s*.+')

    with open(lp_path, 'r') as f:
        for raw_line in f:
            line = raw_line.strip()
            if not line:
                continue

            # ¿Empezamos la sección de restricciones?
            if start_pattern.match(line):
                in_subject_to = True
                continue

            # ¿Terminamos la sección?
            if in_subject_to and end_pattern.match(line):
                break

            # Si estamos dentro de Subject To, intentamos extraer nombre
            if in_subject_to:
                m = constr_pattern.match(line)
                if m:
                    name = m.group(1)
                    constraints.add(name)
                # Si la línea no tiene "nombre:", será continuación de la anterior → la ignoramos

    return constraints

def find_removed_constraints(original_lp, preprocessed_lp):
    """
    Compara los sets de nombres de restricciones y devuelve la lista
    de las que fueron eliminadas tras el preprocesamiento.
    """
    orig_set = parse_constraints(original_lp)
    pre_set  = parse_constraints(preprocessed_lp)

    removed = sorted(orig_set - pre_set)
    return removed

if __name__ == '__main__':
    archivo_original = '/home/wilmer/Documentos/Codes/tesis/Instancias/test/test1/test1.lp'
    archivo_pre      = '/home/wilmer/Documentos/Codes/tesis/Instancias/test/test1/test1_pre.lp'

    eliminadas = find_removed_constraints(archivo_original, archivo_pre)
    if eliminadas:
        print("Restricciones eliminadas tras el preprocesamiento: ", len(eliminadas))
        for name in eliminadas:
            print(" -", name)
        
    else:
        print("No se detectaron restricciones eliminadas.")

In [ ]:
a = [i for i in indicesw if i not in eliminadas]
len(a)/580